# Introduction

This is a guided project using SQLite and the Chinook database provided in the DataQuest course.

We will be reviewing the data and answering three business questions.

First, we must connect our Jupyter Notebook to our database file.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

We can use this query to get a list of all tables and views in our database.

In [2]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Now that we have reviewed our data, we can start analyzing it.

** Question 1: Which genres sell the most tracks in the USA? **

To answer this, we are first going to create a table showing all tracks purchased from customers in the USA.

Then, we are going to join this table to the track and genre tables and use SUM and GROUP BY to show the quantity of tracks sold per genre.

Using a subquery in the SELECT statement, we can also calculate the proportion of sales per genre.

In [51]:
%%sql

WITH tracks_sold_us AS (

SELECT il.*
FROM invoice_line AS il
JOIN invoice AS i
ON il.invoice_id=i.invoice_id
JOIN customer AS c
ON i.customer_id=c.customer_id
WHERE c.country='USA')

SELECT g.name, SUM(ts.quantity) AS quantity_sold, ROUND(SUM(ts.quantity * 1.0)/
        (SELECT SUM(quantity)
        FROM tracks_sold_us) * 100,2) AS proportion_of_sales
FROM tracks_sold_us AS ts
JOIN track AS t
ON ts.track_id=t.track_id
JOIN genre AS g
ON t.genre_id=g.genre_id
GROUP BY g.name
ORDER BY proportion_of_sales DESC;

Done.


name,quantity_sold,proportion_of_sales
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


We can see that Rock, Alternative & Punk, and Metal are the top three genres.

Out of the four albums we were given by our business partners, Red Tone, Meteor and the Girls, and Slim Jim Bites would be the best album choices to add to the store because their genres (Punk, Pop, and Blues) have a higher proporation of sales than Hip Hop, the genre of the fourth album.

That said, since Rock is the highest selling genre by far with 53% of total US sales, we should investigate whether there are new Rock albums we can add to the store as well.

** Question 2: What is the total dollar amount of sales assigned to each sales support agent within the company? **

To answer this, we can create a table joining the customer and invoice tables and using SUM to find the total sales per customer.

We can then join this table to the employee table to find the employee's name, hire date, and total sales per employee.

In [18]:
%%sql

WITH customer_sales AS (

SELECT c.customer_id, c.support_rep_id, SUM(i.total) AS total_sales
FROM customer AS c
JOIN invoice AS i
ON c.customer_id=i.customer_id
GROUP BY c.customer_id, c.support_rep_id)

SELECT e.employee_id, e.first_name || ' ' || e.last_name AS name,
        e.hire_date, SUM(cs.total_sales) AS total_sales
FROM employee AS e
JOIN customer_sales AS cs
ON e.employee_id=cs.support_rep_id
GROUP BY e.employee_id
ORDER BY total_sales;

Done.


employee_id,name,hire_date,total_sales
5,Steve Johnson,2017-10-17 00:00:00,1393.92
4,Margaret Park,2017-05-03 00:00:00,1584.0000000000002
3,Jane Peacock,2017-04-01 00:00:00,1731.5099999999998


While Steve Johnson has the lowest sales, he's also the newest employee, which needs to be taken into consideration when looking at performance.

** Question 3: What is the sales data by country? **

We start by creating a table that shows number of orders and total sales by customer.

We then join this table to another that groups the information by country.

In [31]:
%%sql

WITH per_customer AS (

SELECT c.customer_id, c.country, COUNT(i.invoice_id) AS no_of_orders, SUM(i.total) AS sales
FROM invoice AS i
JOIN customer AS c
ON i.customer_id=c.customer_id
GROUP BY c.customer_id)

SELECT pc.country, COUNT(pc.customer_id) AS no_of_customers,
        SUM(pc.no_of_orders) AS no_of_orders, SUM(pc.sales) AS total_sales,
        SUM(pc.sales*1.0)/COUNT(pc.customer_id) AS avg_sales_per_customer,
        SUM(pc.sales*1.0)/SUM(pc.no_of_orders) AS avg_order_value
FROM per_customer AS pc
GROUP BY pc.country
ORDER BY pc.country;

Done.


country,no_of_customers,no_of_orders,total_sales,avg_sales_per_customer,avg_order_value
Argentina,1,5,39.6,39.6,7.92
Australia,1,10,81.18,81.18,8.118
Austria,1,9,69.3,69.3,7.699999999999999
Belgium,1,7,60.38999999999999,60.38999999999999,8.627142857142855
Brazil,5,61,427.67999999999995,85.53599999999999,7.011147540983606
Canada,8,76,535.59,66.94875,7.047236842105264
Chile,1,13,97.02000000000001,97.02000000000001,7.463076923076924
Czech Republic,2,30,273.24,136.62,9.108
Denmark,1,10,37.61999999999999,37.61999999999999,3.761999999999999
Finland,1,11,79.2,79.2,7.2


We can see that several countries only have one customer. We now want to group these countries into an "Other" category instead of listing them separately.

We can do this using a CASE statement.

In [49]:
%%sql

WITH sales_per_country AS (

SELECT

(CASE
    WHEN (SELECT COUNT(*)
          FROM customer
          WHERE country=c.country) = 1
          THEN 'Other'
          ELSE c.country
 END) AS country_or_other,
        COUNT(DISTINCT i.customer_id) AS no_of_customers,
        COUNT(i.invoice_id) AS no_of_orders,
        SUM(i.total) AS total_sales
FROM customer AS c
JOIN invoice AS i
ON c.customer_id=i.customer_id
GROUP by country_or_other)

SELECT sp.country_or_other, sp.no_of_customers, sp.no_of_orders, sp.total_sales,
       sp.total_sales*1.0/sp.no_of_customers AS avg_sales_per_customer,
       sp.total_sales*1.0/sp.no_of_orders AS avg_order_value
FROM sales_per_country AS sp
ORDER BY country_or_other;

Done.


country_or_other,no_of_customers,no_of_orders,total_sales,avg_sales_per_customer,avg_order_value
Brazil,5,61,427.68000000000006,85.53600000000002,7.0111475409836075
Canada,8,76,535.5900000000001,66.94875000000002,7.047236842105265
Czech Republic,2,30,273.24000000000007,136.62000000000003,9.108000000000002
France,5,50,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,41,334.62,83.655,8.161463414634147
India,2,21,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,147,1094.9400000000003,72.99600000000002,7.44857142857143
Portugal,2,29,185.13000000000002,92.56500000000001,6.383793103448276
USA,13,131,1040.4899999999998,80.0376923076923,7.942671755725189
United Kingdom,3,28,245.52,81.84,8.768571428571429
